### Low order + All order

In [1]:
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import GPUtil

import torch
import gpytorch

from scipy.stats import pearsonr
from scipy.special import binom as binom
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [2]:
import sys
sys.path.insert(1, '../')
import EpiK.models as models

In [3]:
from EpiK.functions import get_data, get_envs, set_data_path
set_data_path("../matsui_data/")

In [4]:
# training sizes
props = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95]

In [5]:
# check_point sizes
partitions = [2, 2, 2, 2, 4, 4, 4, 4, 16, 32, 64, 120]
pd.DataFrame({"props":props, "partitions":partitions}).to_csv("partition_sizes.csv", index=None)

In [6]:
partition_sizes = pd.read_csv("partition_sizes.csv")

### Data

In [7]:
env_list = get_envs()
env = env_list[5]

In [9]:
data_path = "../matsui_data/"

In [10]:
df = pd.read_csv(data_path + "96ghpptzvf-4/SData2/"+ env + "_geno.txt", sep='\t', nrows=5, engine='python')
ids = list(df.columns[3:])

In [12]:
geno_t = torch.load(data_path + env + '_matsui_geno_t.pt', map_location=torch.device('cpu'))

In [13]:
geno_t = torch.tensor(geno_t, dtype=torch.float)
geno_t = torch.transpose(geno_t, 0, 1)
N, L = geno_t.shape

/scratch/local/48134653/ipykernel_12360/1784263641.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  geno_t = torch.tensor(geno_t, dtype=torch.float)


In [14]:
geno_np = geno_t.numpy()

In [15]:
geno_df = pd.DataFrame(geno_np)

In [18]:
geno_df.to_csv(data_path + "NaCl2_geno.csv", index=None)